In [1]:

import os
import pickle
import time
import datetime
from datetime import date
import numpy as np
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from bs4 import BeautifulSoup
from bs4 import BeautifulSoup
from dotenv import load_dotenv
load_dotenv()


C:\Users\yoith\AppData\Local\Temp\ipykernel_13296\109443032.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


True

# LinkedIn Job Scraper: Extracts job details and links from LinkedIn job search pages

In [12]:
def scrape_linkedin_jobs(num_pages=15):
    """
    Scrapes LinkedIn job search pages for specific keywords and locations.
    
    :param num_pages: Number of job search pages to scrape (default 25).
    :return: List of BeautifulSoup objects for each job page scraped, and a Data Frame with all the links of job pages
    """
    # Keywords and Locations
    keywords = [
        "Data Analyst",
        "Data Engineer",
        "Data Scientist"
    ]

    locations = [
        "Spain",
        "England",
        "Scotland",
    ]
    
    # Get login credentials
    email = os.getenv("email")
    password = os.getenv("password")
    
    # WebDriver setup
    options = webdriver.ChromeOptions() 
    options.add_argument('--incognito')
    driver = webdriver.Chrome(options=options)
    driver.get("https://www.linkedin.com/login")
    time.sleep(random.randint(1, 3))  
    
    # Fill login credentials
    email_box = driver.find_element(By.ID, "username")
    password_box = driver.find_element(By.ID, "password")
    email_box.send_keys(email)
    password_box.send_keys(password)
    password_box.send_keys(Keys.RETURN)
    time.sleep(random.randint(1, 7))  

    # List to store BeautifulSoup objects of job pages
    soup_list = []

    # List to store dictionaries of job details
    job_details = []

    for keyword in keywords:
        for location in locations:
            # Build LinkedIn job search URL
            base_url = f"https://www.linkedin.com/jobs/search/?keywords={keyword}&location={location}&start="

            new_soup_list = []

            for page in range(num_pages):
                page_url = base_url + str(page * 25)
                driver.get(page_url)
                time.sleep(random.uniform(3, 7))

                # Extract job links
                links = driver.find_elements(By.CSS_SELECTOR, "a.job-card-container__link.job-card-list__title")
                job_links = [link.get_attribute("href") for link in links]

                for job_link in job_links:
                    driver.get(job_link)
                    time.sleep(random.uniform(3, 7))
                    html = driver.execute_script("return document.body.outerHTML;")
                    soup = BeautifulSoup(html, "html.parser")
                    new_soup_list.append(soup)  # Append current soup to the list

                    # Append job details to list of dictionaries
                    job_details.append({'Keyword': keyword, 'Location': location, 'Job_Link': job_link})

            print(f"For '{keyword}' in '{location}', were found {len(new_soup_list)} results.")
            soup_list.extend(new_soup_list)

     # Save soup_list using pickle
    with open('soup_list.pkl', 'wb') as f:
        pickle.dump(soup_list, f)
    
    driver.quit()

    # Convert list of dictionaries to DataFrame and save it in a .csv
    df_links = pd.DataFrame(job_details)
    df_links.to_csv("data/links_and_query.csv", index=False)
    
    return soup_list, df_links

# Execute the function
soup_list, df_links = scrape_linkedin_jobs(num_pages=15)

For 'Data Analyst' in 'Spain', were found 105 results.
For 'Data Analyst' in 'England', were found 105 results.
For 'Data Analyst' in 'Scotland', were found 42 results.
For 'Data Engineer' in 'Spain', were found 105 results.
For 'Data Engineer' in 'England', were found 105 results.
For 'Data Engineer' in 'Scotland', were found 52 results.
For 'Data Scientist' in 'Spain', were found 105 results.
For 'Data Scientist' in 'England', were found 105 results.
For 'Data Scientist' in 'Scotland', were found 14 results.


# Load Soup List from Pickle File

In [126]:
# This part of the code, load the file with the soup information in case of need it

"""with open('soup_list.pkl', 'rb') as f:
soup_list = pickle.load(f)"""


"with open('soup_list.pkl', 'rb') as f:\nsoup_list = pickle.load(f)"

# LinkedIn Job Offer Extractor: Extracts job details from BeautifulSoup objects and saves to a CSV file

In [123]:
def extract_job_info(soup_list):
    """
    Extract job information from a list of Beautiful Soup objects and save it to a CSV file.

    Args:
        soup_list (list): List of Beautiful Soup objects containing job offer information.

    Returns:
        pandas.DataFrame: DataFrame containing job offer information.
    """
    
    # List to store job offer dictionaries
    offers = []

    # Iterate through each job offer
    for soup in soup_list:
        # Dictionary to store information for each job offer
        offer_dict = {
            "title": np.nan,
            "company": np.nan,
            "link": np.nan,
            "c_": np.nan,
            "l_": np.nan,
            "posted_date": np.nan,  # Changing column name from a_ to posted_date
            "applications": np.nan,
            "job_modality": np.nan,
            "f_p_time": np.nan,
            "job_responsibility": np.nan,
            "job_skills": np.nan,
            "job_salary": np.nan,
            "scraped_on": date.today()
        }

        try:
            # Extracting company name and job title
            company = soup.find('div', class_='p5')
            title = soup.find('h1', class_='t-24 t-bold job-details-jobs-unified-top-card__job-title').getText()
            sub = soup.find('div', class_='job-details-jobs-unified-top-card__primary-description-container')

            # Extracting link to the job posting
            if sub.find("a"):
                link = sub.find("a").get("href")
                offer_dict["link"] = link

            # Extracting company name
            if sub.find("a"):
                company_name = sub.find("a").getText()
                offer_dict["company"] = company_name

            # Extracting more job details
            more_info = sub.getText().strip().split("·")

            c_ = more_info[0].strip()
            l_ = more_info[1].strip()
            posted_date = more_info[2].strip()  # Changing variable name from a_ to posted_date

            offer_dict["title"] = title
            offer_dict["c_"] = c_
            offer_dict["l_"] = l_
            offer_dict["posted_date"] = posted_date  # Changing column name from a_ to posted_date

            # Adding job application count if available
            applications = more_info[3].strip()  # Changing variable name from s_ to applications
            offer_dict["applications"] = applications  # Changing column name from s_ to applications

            # Adding job modality if available
            modality = soup.find('span', class_='ui-label--accent-3')
            if modality:
                job_modality = modality.text.strip()
                offer_dict["job_modality"] = job_modality

            # Adding information about full/part-time status if available
            full_part_time = soup.find('span', class_="job-details-jobs-unified-top-card__job-insight-view-model-secondary")
            if full_part_time:
                f_p_time = full_part_time.text.strip()
                offer_dict["f_p_time"] = f_p_time

            # Adding job responsibilities if available
            responsability = soup.find("li", class_="job-details-jobs-unified-top-card__job-insight job-details-jobs-unified-top-card__job-insight--highlight")
            if responsability:
                job_responsibility = responsability.text.strip().split("\n")[-1]
                offer_dict["job_responsibility"] = job_responsibility

            # Adding required job skills if available
            skills = soup.find_all('a', class_='job-details-how-you-match__skills-item-subtitle')
            if skills:
                job_skills = [skill.text.strip() for skill in skills]
                offer_dict["job_skills"] = ', '.join(job_skills)

            # Adding job salary if available
            salary = soup.find('p', class_='t-16')
            if salary:
                job_salary = salary.getText().strip()
                offer_dict["job_salary"] = job_salary

            # Append the offer dictionary to the offers list if it's not a duplicate
            if offer_dict not in offers:
                offers.append(offer_dict)
        except Exception as e:
            print("Error:", e)

    # Creating a DataFrame from the offers list
    df_all_info = pd.DataFrame(offers)
    
    # Saving DataFrame to a CSV file
    df_all_info.to_csv("data/df_all_info.csv", index=False)

    return df_all_info


Error: 'NoneType' object has no attribute 'getText'
Error: 'NoneType' object has no attribute 'getText'
Error: 'NoneType' object has no attribute 'getText'
Error: 'NoneType' object has no attribute 'getText'
Error: 'NoneType' object has no attribute 'getText'
Error: 'NoneType' object has no attribute 'getText'
Error: 'NoneType' object has no attribute 'getText'
Error: 'NoneType' object has no attribute 'getText'
Error: 'NoneType' object has no attribute 'getText'
Error: list index out of range


# Modifications

In [5]:
"""fer"""

def selenium_driver ():
    # Get login credentials
    email = os.getenv("email")
    password = os.getenv("password")
    
    # WebDriver setup
    options = webdriver.ChromeOptions() 
    options.add_argument('--incognito')
    driver = webdriver.Chrome(options=options)
    driver.get("https://www.linkedin.com/login")
    time.sleep(random.randint(1, 3))  
    
    email_box = driver.find_element(By.ID, "username")
    password_box = driver.find_element(By.ID, "password")
    email_box.send_keys(email)
    password_box.send_keys(password)
    password_box.send_keys(Keys.RETURN)
    time.sleep(random.randint(1, 7))  

    return driver

def get_links(driver, num_pages=25):
    """
    Scrapes LinkedIn job search pages for specific keywords and locations.
    
    :param num_pages: Number of job search pages to scrape (default 25).
    :return: List of BeautifulSoup objects for each job page scraped, and a Data Frame with all the links of job pages
    """
    # Keywords and Locations
    keywords = [
        "Data Analyst",
        "Data Engineer",
        "Data Scientist"
    ]

    locations = [
        "Spain",
        "England",
        "Scotland",
    ]
    
    
    # List to store BeautifulSoup objects of job pages
    soup_list = []

    # DataFrame to store job links with additional explanations
    #df_links = pd.DataFrame(columns=['Keyword', 'Location', 'Job_Link'])
    new_list = []
    for keyword in keywords:
        for location in locations:
            # Build LinkedIn job search URL
            base_url = f"https://www.linkedin.com/jobs/search/?keywords={keyword}&location={location}&start="

            new_soup_list = []

            for page in range(num_pages):
                page_url = base_url + str(page * 25)
                driver.get(page_url)
                time.sleep(random.uniform(3, 7))

                # Extract job links
                links = driver.find_elements(By.CSS_SELECTOR, "a.job-card-container__link.job-card-list__title")
                job_links = [link.get_attribute("href") for link in links]
                for job_link in job_links:
                    dict_ = {'Keyword': keyword, 
                                                'Location': location, 
                                                'Job_Link': job_link}
                    if dict_ not in new_list:
                        new_list.append(dict_)
    df_links_search = pd.DataFrame(new_list)
    df_links_search.to_csv("links_and_query.csv", index=False)
    return df_links_search

              
def get_html_from_link (link, driver):
   
    driver.get(link)
    time.sleep(random.randint(1, 3))
    html = driver.execute_script("return document.body.outerHTML;")
    soup = BeautifulSoup(html, "html.parser")
    return soup

def add_html_column (driver):
    df_links_search["html"] = df_links_search["Job_Link"].apply(lambda link: get_html_from_link(link, driver=driver))
    return df_links_search

def scrape_job_details(soup):
    """
    Extracts job details from a list of BeautifulSoup objects.

    :param soup: List of BeautifulSoup objects for each job page scraped.
    :return: DataFrame containing extracted job details.
    
    """
    oferta = []  # Initialize an empty list to store job details
     # Iterate over each BeautifulSoup object in soup_list
    
    

    # Extract company information
    company = soup.find('div', class_='p5')
    title = company.find("h1", {"class": "t-24 t-bold job-details-jobs-unified-top-card__job-title"}).getText()
    sub = company.find("div", {"class": "job-details-jobs-unified-top-card__primary-description-container"})
    modality = soup.find('li', class_='job-details-jobs-unified-top-card__job-insight job-details-jobs-unified-top-card__job-insight--highlight').getText()
    modality_job = modality.strip().split("La modalidad laboral es ")[1].split(".")[0]
    salary = np.nan
   

    # Extract job link and company name if available
    if sub.find("a"):
        link = sub.find("a").get("href")

    if sub.find("a"):
        empresa = sub.find("a").getText()
        

    # Extract additional information
    more_info = sub.getText().strip().split("·")

    c_ = more_info[0].strip()  # Company name
    l_ = more_info[1].strip()  # Location

    a_ = more_info[2].strip()  # Employment type
    s_ = more_info[3].strip()  # Experience level

    scraped_on = today = date.today()  # Add scraped date

    # Append the dictionary to the list
    if dict_ not in oferta:
        oferta.append(dict_)

    return title, empresa, link, c_, l_, a_, s_, scraped_on, modality_job, salary



def all_together (driver, num_pages=25):
    
    #1. Build links and query dataset
    #df_links_search = get_links(driver, num_pages=num_pages)
    df_links_search = pd.read_csv("links_and_query.csv")
    
    #2. Get html from each link
    df_links_search = add_html_column (driver)
    driver.quit()
   
    #3. Get info from each html
    df_links_search[["title", "company", "link", "c_", "l_", "a_", "s_", "scraped_on", "modality_job", "salary"]] = df_links_search["html"].apply(scrape_job_details)
    
    # 4. Export
    df_links_search.to_csv("todojunto.csv", index=False)
    return df_links_search

driver = selenium_driver()
df = all_together(driver, num_pages=10)
df

NameError: name 'df_links_search' is not defined

# First Modification

```python
"""fer"""

def selenium_driver ():
    # Get login credentials
    email = os.getenv("email")
    password = os.getenv("password")
    
    # WebDriver setup
    options = webdriver.ChromeOptions() 
    options.add_argument('--incognito')
    driver = webdriver.Chrome(options=options)
    driver.get("https://www.linkedin.com/login")
    time.sleep(random.randint(1, 3))  
    
    email_box = driver.find_element(By.ID, "username")
    password_box = driver.find_element(By.ID, "password")
    email_box.send_keys(email)
    password_box.send_keys(password)
    password_box.send_keys(Keys.RETURN)
    time.sleep(random.randint(1, 7))  

    return driver

def get_links(driver, num_pages=25):
    """
    Scrapes LinkedIn job search pages for specific keywords and locations.
    
    :param num_pages: Number of job search pages to scrape (default 25).
    :return: List of BeautifulSoup objects for each job page scraped, and a Data Frame with all the links of job pages
    """
    # Keywords and Locations
    keywords = [
        "Data Analyst",
        "Data Engineer",
        "Data Scientist"
    ]

    locations = [
        "Spain",
        "England",
        "Scotland",
    ]
    
    

    # List to store BeautifulSoup objects of job pages
    soup_list = []

    # DataFrame to store job links with additional explanations
    #df_links = pd.DataFrame(columns=['Keyword', 'Location', 'Job_Link'])
    new_list = []
    for keyword in keywords:
        for location in locations:
            # Build LinkedIn job search URL
            base_url = f"https://www.linkedin.com/jobs/search/?keywords={keyword}&location={location}&start="

            new_soup_list = []

            for page in range(num_pages):
                page_url = base_url + str(page * 25)
                driver.get(page_url)
                time.sleep(random.uniform(3, 7))

                # Extract job links
                links = driver.find_elements(By.CSS_SELECTOR, "a.job-card-container__link.job-card-list__title")
                job_links = [link.get_attribute("href") for link in links]
                for job_link in job_links:
                    dict_ = {'Keyword': keyword, 
                                                'Location': location, 
                                                'Job_Link': job_link}
                    if dict_ not in new_list:
                        new_list.append(dict_)
    df_links_search = pd.DataFrame(new_list)
    return df_links_search

              
def get_html_from_link (driver, link):
   
    driver.getlink(link)
    time.sleep(random.randint(1, 3))
    html = driver.execute_script("return document.body.outerHTML;")
    soup = BeautifulSoup(html, "html.parser")
    return soup

def scrape_job_details(soup):
    """
    Extracts job details from a list of BeautifulSoup objects.

    :param soup_list: List of BeautifulSoup objects for each job page scraped.
    :return: DataFrame containing extracted job details.
    
    """
    oferta = []  # Initialize an empty list to store job details

     # Iterate over each BeautifulSoup object in soup_list
    dict_ = {  # Create a dictionary to store job details with default values as NaN
        "title": np.nan,
        "company": np.nan,
        "link": np.nan,
        "c_": np.nan,
        "l_": np.nan,
        "a_": np.nan,
        "s_": np.nan
    }

    # Extract company information
    company = soup.find('div', class_='p5')
    title = company.find("h1", {"class": "t-24 t-bold job-details-jobs-unified-top-card__job-title"}).getText()
    sub = company.find("div", {"class": "job-details-jobs-unified-top-card__primary-description-container"})

    # Extract job link and company name if available
    if sub.find("a"):
        link = sub.find("a").get("href")
        dict_["link"] = link

    if sub.find("a"):
        empresa = sub.find("a").getText()
        dict_["company"] = empresa

    # Extract additional information (location, employment type, experience level, and job function)
    more_info = sub.getText().strip().split("·")

    c_ = more_info[0].strip()  # Company name
    l_ = more_info[1].strip()  # Location

    a_ = more_info[2].strip()  # Employment type
    s_ = more_info[3].strip()  # Experience level

    # Update the dictionary with extracted job details
    dict_["title"] = title
    dict_["c_"] = c_
    dict_["l_"] = l_
    dict_["a_"] = a_
    dict_["s_"] = s_

    dict_["scraped_on"] = today = date.today()  # Add scraped date

    # Append the dictionary to the list
    if dict_ not in oferta:
        oferta.append(dict_)

    # Create a DataFrame from the list of job details
    df = pd.DataFrame(oferta)
    return df

driver = selenium_driver()
df_links_search = get_links(driver, num_pages=25)
df_links_search["html"] = df["job_link"].apply(get_html_from_link(driver))
df_links_search[["title", "company", "link", "c_", "l_", "a_", "s_"]] = df["job_link"].apply(scrape_job_details, axis=1)
driver.quit()
df.to_csv("todojunto.csv", index=False)
df
```

#### 